In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['patch.force_edgecolor'] = True
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [5]:
filepath = "../dataset/"
file= filepath +'training_data.csv'

training_data = pd.read_csv(file)

bag_of_words_bot = 'bot|b0t'

# Performing Feature Engineering

In [6]:
training_data['screen_name_binary'] = training_data.screen_name.str.contains(bag_of_words_bot, case=False, na=False)
training_data['name_binary'] = training_data.name.str.contains(bag_of_words_bot, case=False, na=False)
training_data['description_binary'] = training_data.description.str.contains(bag_of_words_bot, case=False, na=False)
training_data['status_binary'] = training_data.status.str.contains(bag_of_words_bot, case=False, na=False)
training_data['listed_count_binary'] = (training_data.listed_count>20000)==False

features = ['screen_name_binary', 'name_binary', 'description_binary', 'status_binary', 'verified', 'followers_count', 'friends_count', 'statuses_count', 'listed_count_binary', 'bot']

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

y = training_data.bot
X = training_data[features].iloc[:,:-1]

# Implementing Different Models

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier


classificators = [
    DecisionTreeClassifier(),
    MultinomialNB(),
    RandomForestClassifier(),
    LinearSVC(),
    LogisticRegression(),
    GaussianNB(),
    KNeighborsClassifier(),
    GradientBoostingClassifier(),
    Perceptron()
]

In [17]:
for model in classificators:
    print(model.__class__.__name__)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

    model = model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    print("Trainig Accuracy: %.5f" %accuracy_score(y_train, y_pred_train))
    print("Test Accuracy: %.5f" %accuracy_score(y_test, y_pred_test))
    print("Confusion matrix: ")
    print(confusion_matrix(y_test, y_pred_test))
    print("Classification report: ")
    print(classification_report(y_test, y_pred_test))
    print()

DecisionTreeClassifier
Trainig Accuracy: 0.99949
Test Accuracy: 0.86310
Confusion matrix: 
[[377  46]
 [ 69 348]]
Classification report: 
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       423
           1       0.88      0.83      0.86       417

   micro avg       0.86      0.86      0.86       840
   macro avg       0.86      0.86      0.86       840
weighted avg       0.86      0.86      0.86       840


MultinomialNB
Trainig Accuracy: 0.67910
Test Accuracy: 0.69762
Confusion matrix: 
[[181 242]
 [ 12 405]]
Classification report: 
              precision    recall  f1-score   support

           0       0.94      0.43      0.59       423
           1       0.63      0.97      0.76       417

   micro avg       0.70      0.70      0.70       840
   macro avg       0.78      0.70      0.67       840
weighted avg       0.78      0.70      0.67       840


RandomForestClassifier
Trainig Accuracy: 0.99080
Test Accuracy: 0.89048
Confu